In [1]:
import pandas as pd
import numpy as np

In [2]:
# This will allow you to see all column names & rows when you are doing .head(). None of the column name will be truncated.
# source: https://stackoverflow.com/questions/49188960/how-to-show-all-of-columns-name-on-pandas-dataframe

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [3]:
# source: https://gist.github.com/rozanecm/38f2901c592bdffc40726cb0473318cf
# Function which plays a beep of given duration and frequency.
# Useful for when executing things that need a while to finish, to get notified.
import os
def beep(duration = 1, freq = 1500):
    """ play tone of duration in seconds and freq in Hz. """
    os.system('play --no-show-progress --null --channels 1 synth %s sine %f' % (duration, freq))

In [36]:
train = pd.read_csv('../data/train.csv', dtype={'tipodepropiedad':'category', 'ciudad':'category', 'provincia':'category', 'id':'int32', 'antiguedad':'float16', 'habitaciones':'float16', 'garages':'float16', 'banos':'float16', 'metroscubiertos':'float16', 'metrostotales':'float16', 'idzona':'float16', 'lat':'float16', 'lng':'float16', 'gimnasio':'bool', 'usosmultiples':'bool', 'piscina':'bool', 'escuelascercanas':'bool', 'centroscomercialescercanos':'bool'}, parse_dates=['fecha'])
test = pd.read_csv('../data/test.csv', dtype={'tipodepropiedad':'category', 'ciudad':'category', 'provincia':'category', 'id':'int32', 'antiguedad':'float16', 'habitaciones':'float16', 'garages':'float16', 'banos':'float16', 'metroscubiertos':'float16', 'metrostotales':'float16', 'idzona':'float16', 'lat':'float16', 'lng':'float16', 'gimnasio':'bool', 'usosmultiples':'bool', 'piscina':'bool', 'escuelascercanas':'bool', 'centroscomercialescercanos':'bool'}, parse_dates=['fecha'])

In [5]:
rescued_coords = pd.read_csv('../data/rescueLatLongs.csv')
rescued_antiguedad = pd.read_csv('../data/imputations/antiguedad.csv')
rescued_banos = pd.read_csv('../data/imputations/banos.csv')
rescued_garages = pd.read_csv('../data/imputations/garages.csv')
rescued_habitaciones = pd.read_csv('../data/imputations/habitaciones.csv')
rescued_metroscubiertos = pd.read_csv('../data/imputations/metroscubiertos.csv')
rescued_metrostotales = pd.read_csv('../data/imputations/metrostotales.csv')

In [6]:
# mergeamos con coords. extra obtenidas en tp1.
train = train.merge(rescued_coords.drop('Unnamed: 0', axis=1), how='left', on='id')
train['lat_x'] = train.apply(lambda x: x['lat_y'] if pd.isna(x['lat_x']) else x['lat_x'], axis=1)
train['lng_x'] = train.apply(lambda x: x['lng_y'] if pd.isna(x['lng_x']) else x['lng_x'], axis=1)
train.drop(['lat_y','lng_y'], axis=1, inplace=True)
train.rename(columns={'lat_x':'lat','lng_x':'lon'}, inplace=True)

# por consistencia, para que ambos datasets tengan mismos nombres
test.rename(columns={'lng':'lon'}, inplace=True)
train.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lon,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,2.0,80.0,80.0,23536.0,31.733330,-106.48333,2015-08-23,False,False,False,False,False,2273000.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,2.0,268.0,180.0,24512.0,19.312500,-99.25000,2013-06-28,False,False,False,True,True,3600000.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,2.0,144.0,166.0,48544.0,NaN,NaN,2015-10-17,False,False,False,False,False,1200000.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,1.0,63.0,67.0,53664.0,19.296875,-99.68750,2012-03-09,False,False,False,True,True,650000.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,1.0,95.0,95.0,47840.0,20.723560,-103.38479,2016-06-07,False,False,False,False,False,1150000.0


In [7]:
def fillna_with_models_predictions(df, predictions_df, col_name):
    indicadora_name = "tiene_" + col_name
    df[indicadora_name] = df[col_name].notna()
    
    df = df.merge(predictions_df, how='left', on='id')
    original_col = col_name + "_x"
    filler_col = col_name + "_y"
    df[col_name] = df.apply(lambda x: x[filler_col] if pd.isna(x[original_col]) else x[original_col], axis=1)
    df.drop([original_col,filler_col], axis=1, inplace=True)
    
    return df

In [8]:
train = fillna_with_models_predictions(train, rescued_antiguedad, 'antiguedad')
train = fillna_with_models_predictions(train, rescued_banos, 'banos')
train = fillna_with_models_predictions(train, rescued_garages, 'garages')
train = fillna_with_models_predictions(train, rescued_habitaciones, 'habitaciones')
train = fillna_with_models_predictions(train, rescued_metroscubiertos, 'metroscubiertos')
train = fillna_with_models_predictions(train, rescued_metrostotales, 'metrostotales')

test = fillna_with_models_predictions(test, rescued_antiguedad, 'antiguedad')
test = fillna_with_models_predictions(test, rescued_banos, 'banos')
test = fillna_with_models_predictions(test, rescued_garages, 'garages')
test = fillna_with_models_predictions(test, rescued_habitaciones, 'habitaciones')
test = fillna_with_models_predictions(test, rescued_metroscubiertos, 'metroscubiertos')
test = fillna_with_models_predictions(test, rescued_metrostotales, 'metrostotales')

In [9]:
# Nan para los datos fuera de rango es mejor que dropear todo el dato
train.loc[(train['lat']>14) | (train['lat']<33),['lat','lon']] = np.nan
train.loc[(train['lon']>86) | (train['lon']<118),['lat','lon']] = np.nan

In [10]:
# inf. values don't make sense. I think it's preferable to treat them as nans directly.
train.replace([np.inf, -np.inf], np.nan, inplace=True)

In [11]:
test.loc[(train['lat']>14) | (train['lat']<33),['lat','lon']] = np.nan
test.loc[(train['lon']>86) | (train['lon']<118),['lat','lon']] = np.nan

In [12]:
test.replace([np.inf, -np.inf], np.nan, inplace=True)

# Agregamos features que ya hemos creado para analisis de tp1

In [13]:
def contar_amenities(row):
    return row['gimnasio'] + row['usosmultiples'] + row['piscina'] + row['escuelascercanas'] + row['centroscomercialescercanos']

In [14]:
train['cant_amenities'] = train.apply(lambda x: contar_amenities(x), axis=1)
test['cant_amenities'] = test.apply(lambda x: contar_amenities(x), axis=1)

In [15]:
def feature_fechas(df):
    # Para entender lo de los senos y cosenos: https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/
    df['year'] = df['fecha'].dt.year
    df['month'] = df['fecha'].dt.month
    df['day'] = df['fecha'].dt.day
    df['sin_month'] = np.sin(2*np.pi*df['month']/12)
    df['cos_month'] = np.cos(2*np.pi*df['month']/12)
    # tomo cant. de dias en mes: 31 en todos los casos. Para esto deberia servir bastante bien igual.
    df['sin_day'] = np.sin(2*np.pi*df['day']/31)
    df['cos_day'] = np.cos(2*np.pi*df['day']/31)
    
    # no necesito mas las cols. originales de month y day.
    df.drop(['month','day'], axis=1, inplace=True)
    
feature_fechas(train)
feature_fechas(test)

In [16]:
palabras_avenida = ['avenida', 'av']
train['es_avenida'] = train['direccion'].fillna('no info').apply(lambda x: any(avenida_indicator in x.lower() for avenida_indicator in palabras_avenida))
test['es_avenida'] = test['direccion'].fillna('no info').apply(lambda x: any(avenida_indicator in x.lower() for avenida_indicator in palabras_avenida))

# * Fin agregado de features de tp1 *

El sample submission no tiene header. **Ojo con eso al guardar la submission.** Hagamos la funcion para guardar submissions ahora, para evitar problemas a futuro y despreocuparnos.

In [17]:
# To save predictions.
# There must be a directory ../predictions for this to work as expected.
# source: https://gist.github.com/rozanecm/ee8333741db42b10158b3e0aff3f22aa
import time
def _get_filename(my_name, timestamp):
    return "../predictions/" + timestamp + " by " + my_name + ".csv"

def _save_description(authors_name, timestamp, submission_description):
    f = open("../predictions/" + authors_name + ".txt","a")
    f.write(timestamp + ": " + submission_description + '\n')
    f.close()

def save_submission(submission_df, authors_name="fcozza", description = "no description.", index=False, header=True):
    timestamp = time.strftime("%Y.%m.%d - %H:%M:%S")
    submission_df.to_csv(_get_filename(authors_name, timestamp), index=index, header=header)
    _save_description(authors_name, timestamp, description)

In [18]:
# Define a seed, so all algorithms that accept a seed, take the same, for consistency reasons,
# so everything can be replicated without problems random state
seed=42

In [19]:
X = train.drop('precio', axis=1) #set de datos
y = train['precio'] #target

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=seed)

***

In [21]:
X_train.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,idzona,lat,lon,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,tiene_antiguedad,antiguedad,tiene_banos,banos,tiene_garages,garages,tiene_habitaciones,habitaciones,tiene_metroscubiertos,metroscubiertos,tiene_metrostotales,metrostotales,cant_amenities,year,sin_month,cos_month,sin_day,cos_day,es_avenida
96835,88522,tres lagos moderno departamento,<p>funcional departamento en un nuevo concepto...,Apartamento,"Edif. Bruselas, Laguna de Mayrán No. 166, Tres...",Miguel Hidalgo,Distrito Federal,NaN,NaN,NaN,2013-10-11,True,True,True,True,True,True,8.0000,True,2.0,True,2.000000,True,2.0,True,80.0,True,98.000000,5,2013,-0.866025,5.000000e-01,0.790776,-0.612106,False
213906,45078,venta casa ciudad brisa remodelada cerca escue...,hermosa casa remodelada con muy buenos espacio...,Casa,CIRCUNVALACION PONIENTE 43,Naucalpan de Juárez,Edo. de México,55904.0,NaN,NaN,2015-09-29,False,False,False,True,True,True,10.0000,True,3.0,True,3.000000,True,4.0,True,290.0,True,290.000000,2,2015,-1.000000,-1.836970e-16,-0.394356,0.918958,False
120615,37930,quinta en venta en monte bello,"id:45947, hermosa quinta con casa de 100 mts2 ...",Quinta Vacacional,NaN,Juárez,Nuevo León,NaN,NaN,NaN,2016-04-24,False,False,False,False,False,False,7.7196,True,1.0,False,0.134836,True,2.0,True,235.0,False,235.285333,0,2016,0.866025,-5.000000e-01,-0.988468,0.151428,False
77141,128563,"bonita casa, dos niveles, 3 rec., 3 est., 20 m...","bonita casa en dos plantas, en planta baja, 3 ...",Casa,-,Tlalnepantla de Baz,Edo. de México,56224.0,NaN,NaN,2015-10-20,False,False,False,True,True,True,20.0000,True,2.0,True,3.000000,True,3.0,True,270.0,False,220.016157,2,2015,-0.866025,5.000000e-01,-0.790776,-0.612106,False
109102,152060,"casa en venta en nuestra señora de fatima, sal...",<p>fracc. fatima residencial ubicado en car.. ...,Casa,Santa Barbara 155,Saltillo,Coahuila,5864.0,NaN,NaN,2013-09-25,False,False,False,True,True,True,15.0000,True,2.0,True,2.000000,True,3.0,True,185.0,True,200.000000,2,2013,-1.000000,-1.836970e-16,-0.937752,0.347305,False


# Approach 1 - XGBoost con CV basico (Grid Search)

## Entrenamiento local

In [22]:
X_train = X_train.drop(['id','fecha','titulo', 'descripcion', 'direccion'],axis=1)

In [23]:
#Esto es importante, convierto los datos categoricos en numericos cada categoria una columna con 1 si true o 0 si false
##https://stackoverflow.com/questions/34265102/xgboost-categorical-variables-dummification-vs-encoding
##https://machinelearningmastery.com/data-preparation-gradient-boosting-xgboost-python/
X_train_2 = X_train.join(pd.get_dummies(X_train.ciudad))
X_train_2 = X_train.join(pd.get_dummies(X_train.tipodepropiedad))
X_train_2 = X_train.join(pd.get_dummies(X_train.provincia))

X_train_2.drop('ciudad',axis=1, inplace=True)
X_train_2.drop('tipodepropiedad',axis=1, inplace=True)
X_train_2.drop('provincia',axis=1, inplace=True)

In [24]:
X_train_2.head()

,idzona,lat,lon,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,tiene_antiguedad,antiguedad,tiene_banos,banos,tiene_garages,garages,tiene_habitaciones,habitaciones,tiene_metroscubiertos,metroscubiertos,tiene_metrostotales,metrostotales,cant_amenities,year,sin_month,cos_month,sin_day,cos_day,es_avenida,Aguascalientes,Baja California Norte,Baja California Sur,Campeche,Chiapas,Chihuahua,Coahuila,Colima,Distrito Federal,Durango,Edo. de México,Guanajuato,Guerrero,Hidalgo,Jalisco,Michoacán,Morelos,Nayarit,Nuevo León,Oaxaca,Puebla,Querétaro,Quintana Roo,San luis Potosí,Sinaloa,Sonora,Tabasco,Tamaulipas,Tlaxcala,Veracruz,Yucatán,Zacatecas
96835,NaN,NaN,NaN,True,True,True,True,True,True,8.0000,True,2.0,True,2.000000,True,2.0,True,80.0,True,98.000000,5,2013,-0.866025,5.000000e-01,0.790776,-0.612106,False,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
213906,55904.0,NaN,NaN,False,False,False,True,True,True,10.0000,True,3.0,True,3.000000,True,4.0,True,290.0,True,290.000000,2,2015,-1.000000,-1.836970e-16,-0.394356,0.918958,False,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
120615,NaN,NaN,NaN,False,False,False,False,False,False,7.7196,True,1.0,False,0.134836,True,2.0,True,235.0,False,235.285333,0,2016,0.866025,-5.000000e-01,-0.988468,0.151428,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
77141,56224.0,NaN,NaN,False,False,False,True,True,True,20.0000,True,2.0,True,3.000000,True,3.0,True,270.0,False,220.016157,2,2015,-0.866025,5.000000e-01,-0.790776,-0.612106,False,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
109102,5864.0,NaN,NaN,False,False,False,True,True,True,15.0000,True,2.0,True,2.000000,True,3.0,True,185.0,True,200.000000,2,2013,-1.000000,-1.836970e-16,-0.937752,0.347305,False,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

params = {  
    "n_estimators": [200],
    "max_depth": [20,22,25],
    "learning_rate": [0.06], 
    "colsample_bytree": [1],
    "subsample": [0.946934], 
    "gamma":[30],
    'reg_alpha': [10],
    "min_child_weight": [11]
}

regXGB = xgb.XGBRegressor(objective ='reg:squarederror',nthread=-1) 

regXGBwithCV = GridSearchCV(regXGB, params, n_jobs=-1,verbose=10,cv=3) # n_iters es la cant de veces que busca, 10 es lo default

regXGBwithCV.fit(X_train_2, y_train, eval_metric="rmse")

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   9 | elapsed:  4.7min remaining: 16.4min
[Parallel(n_jobs=-1)]: Done   3 out of   9 | elapsed:  4.8min remaining:  9.5min
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:  5.0min remaining:  6.3min
[Parallel(n_jobs=-1)]: Done   5 out of   9 | elapsed:  5.1min remaining:  4.1min
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed:  5.2min remaining:  2.6min
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed:  5.5min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  8.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:  8.1min finished
/home/fabrizio/anaconda3/envs/tester/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=-1,
                                    objective='reg:squarederror',
                                    random_stat...
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='warn', n_jobs=-1,
             param_grid={'colsample_bytree': [1], 'gamma': [30],
                         'learning_rate': [0.06], 'max_depth': [20, 22, 25],
      

In [27]:
print(regXGBwithCV.best_score_)
print(regXGBwithCV.best_params_)

0.7556491853178223
{'colsample_bytree': 1, 'gamma': 30, 'learning_rate': 0.06, 'max_depth': 20, 'min_child_weight': 11, 'n_estimators': 200, 'reg_alpha': 10, 'subsample': 0.946934}


In [28]:
beep()

## Entrenamiento con todos los datos para obtener predicciones a subir

In [29]:
X = train.drop(['id','fecha','titulo', 'descripcion', 'direccion','precio'], axis=1) #set de datos
y = train['precio'] #target

In [30]:
#Esto es importante, convierto los datos categoricos en numericos cada categoria una columna con 1 si true o 0 si false
##https://stackoverflow.com/questions/34265102/xgboost-categorical-variables-dummification-vs-encoding
##https://machinelearningmastery.com/data-preparation-gradient-boosting-xgboost-python/
X_2 = X.join(pd.get_dummies(X.ciudad))
X_2 = X.join(pd.get_dummies(X.tipodepropiedad))
X_2 = X.join(pd.get_dummies(X.provincia))

X_2.drop('ciudad',axis=1, inplace=True)
X_2.drop('tipodepropiedad',axis=1, inplace=True)
X_2.drop('provincia',axis=1, inplace=True)

In [31]:
#Entrenar modelo
model = xgb.XGBRegressor(colsample_bytree=1,gamma=30,learning_rate=0.06,max_depth=20,min_child_weight=11,n_estimators=200,\
                 reg_alpha=10,subsample=0.946934,nthread=-1,objective ='reg:squarederror')

model.fit(X_2, y,eval_metric="rmse", verbose=True)

/home/fabrizio/anaconda3/envs/tester/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=30,
             importance_type='gain', learning_rate=0.06, max_delta_step=0,
             max_depth=20, min_child_weight=11, missing=None, n_estimators=200,
             n_jobs=1, nthread=-1, objective='reg:squarederror', random_state=0,
             reg_alpha=10, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.946934, verbosity=1)

In [32]:
test = test.drop(['id','fecha','titulo', 'descripcion', 'direccion'], axis=1)

In [33]:
test_2 = test.join(pd.get_dummies(test.ciudad))
test_2 = test.join(pd.get_dummies(test.tipodepropiedad))
test_2 = test.join(pd.get_dummies(test.provincia))

test_2.drop('ciudad',axis=1, inplace=True)
test_2.drop('tipodepropiedad',axis=1, inplace=True)
test_2.drop('provincia',axis=1, inplace=True)

In [34]:
pred = model.predict(test_2)

In [37]:
df = pd.DataFrame(data={'id':test['id'], 'target':pred})

In [38]:
description = "1st approach_2. XGBoost"
save_submission(df, description=description)

In [39]:
beep()

***